In [3]:
# We might need to submit this script as a job, let's give running it like this a run anyways - As expected, not working - let's first get the firts part to run on script

In [4]:
# Guessing this is where we will be saving the extrcated features, but not very sure yet. Let's follow along the notebook, and see where it leads us to.

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from data_generator import DataGenerator, batch_predict

2025-09-10 13:12:25.380765: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-10 13:12:26.686050: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-10 13:12:26.686118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-10 13:12:26.886314: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-10 13:12:27.193887: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# load the model?

In [5]:
overlapping_model = tf.keras.models.load_model("models/CNN_seq2seq_overlapping_300.keras") 

2025-09-10 12:03:01.045993: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [6]:
overlapping_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 13, None, None, 3)   0         []                            
                             ]                                                                    
                                                                                                  
 time_distributed (TimeDist  (None, 13, 32)               71808     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['time_distributed[0][0]']    
                              (None, 64),                                                   

In [7]:
# locate the test data?

In [8]:
# input features
input_features_loc = 'data/test_input_sub_images/'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [9]:
input_contents

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [10]:
# Running inot a lot of memory issues, therefore the loop is cut into chunks to get and save the required outputs.

In [11]:
try_shape = np.load(os.path.join(input_features_loc, input_contents[0]))

In [12]:
try_shape.shape

(1376, 13, 300, 300, 3)

In [13]:
# test targets
out_targets_loc = 'data/test_out_targets'
out_contents = os.listdir(out_targets_loc)
out_contents.sort()

print(out_contents)

['test_targets_blk_0103.npy', 'test_targets_blk_0104.npy', 'test_targets_blk_0105.npy', 'test_targets_blk_0106.npy', 'test_targets_blk_0201.npy', 'test_targets_blk_0202.npy', 'test_targets_blk_0205.npy', 'test_targets_blk_0206.npy', 'test_targets_blk_0302.npy', 'test_targets_blk_0303.npy', 'test_targets_blk_0304.npy', 'test_targets_blk_0305.npy', 'test_targets_blk_0306.npy']


In [14]:
# batch_size = 32
# test_rmse = []
# test_mae = []
# test_r2 = []
# test_pearsonr = []
# preds = []
# for i in range(len(input_contents)):
#     # load the features
#     test_features = np.load(os.path.join(input_features_loc, input_contents[i]))
#     # load targets
#     test_targets = np.load(os.path.join(out_targets_loc, out_contents[i]))
#     test_data_gen = DataGenerator(test_features, test_targets, batch_size, shuffle=False)
#     # Use the batch predictions to generate the predictions
#     test_preds, test_targets_alt = batch_predict(model, test_data_gen, flatten=True, verbose=True)
#     preds.append(test_preds)
#     print(np.mean(test_targets == test_targets_alt))
#     # compute the test scores, I think we need to flatten these before computing the scores - or can use tf, but the answers are going to be the same
#     test_preds_flatten = test_preds.flatten()
#     test_targets_flatten = test_targets_alt.flatten()
#     mae = mean_absolute_error(test_targets_flatten, test_preds_flatten)
#     test_mae.append(mae)
#     rmse = np.sqrt(mean_squared_error(test_targets_flatten, test_preds_flatten))
#     test_rmse.append(rmse)
#     rsquare = r2_score(test_targets_flatten, test_preds_flatten)
#     test_r2.append(rsquare)
#     pearsonr_score = pearsonr(test_targets_flatten, test_preds_flatten)[0]
#     test_pearsonr.append(pearsonr_score)

In [ ]:
%%time
for_sanity_check = []
for i in range(len(input_contents)):
    # load the np file
    load_np_file = np.load(os.path.join(input_features_loc, input_contents[i]))
    # load targets
    test_targets = np.load(os.path.join(out_targets_loc, out_contents[i]))
    # print shape of the loaded file
    print(load_np_file.shape)
    # predicted_values
    # define the test data generator here
    test_data_gen = DataGenerator(test_features, test_targets, batch_size, shuffle=False)
    # Use the batch predictions to generate the predictions
    test_preds, test_targets_alt = batch_predict(overlapping_model, test_data_gen, flatten=True, verbose=True)
    print(np.mean(test_targets == test_targets_alt))
    print(test_preds.shape)
    for_sanity_check.append(test_preds)
    # save these values?
    # name
    loc_name = 'data/predicted_sequences_from_stage_1/' + 'pred_values_blk_' + input_contents[i].split('.')[0][-4:] + '.npy'
    np.save(loc_name, test_preds)

In [2]:
# We probably can run whatever is follows in the notebook itself. Let's give in a try when we have all the predicted values extracted

In [11]:
# might need to do a batch predict due to the dataset size, let's figure this out - okay, why don't we actually do this and try to write a cleaner code? That might be better for future refernces.

In [12]:
# Nah let it be for now, it would take some time - we could get all the extracted features saved in two runs, I think it's okay for now

In [13]:
# Okay, we have done another sanity check here, which i believe is not necessary, but let's try it anyway until we get a memory error

In [6]:
# perform a sanity check
loc_path = 'data/predicted_sequences_from_stage_1/'

In [7]:
loc_contents = os.listdir(loc_path)
loc_contents.sort()

In [8]:
loc_contents

['pred_values_blk_0103.npy',
 'pred_values_blk_0104.npy',
 'pred_values_blk_0105.npy',
 'pred_values_blk_0106.npy',
 'pred_values_blk_0201.npy',
 'pred_values_blk_0202.npy',
 'pred_values_blk_0205.npy',
 'pred_values_blk_0206.npy',
 'pred_values_blk_0302.npy',
 'pred_values_blk_0303.npy',
 'pred_values_blk_0304.npy',
 'pred_values_blk_0305.npy',
 'pred_values_blk_0306.npy']

In [9]:
# Okay, for this we do need the entire loop to run to get the predicted values - so let's just do the sanity check starting from the 6th file here

In [10]:
# %%time
# for i in range(6, len(loc_contents)):
#     load_stored_preds = np.load(os.path.join(loc_path, loc_contents[i]))
#     print(np.mean(load_stored_preds == for_sanity_check[i-6]))

In [11]:
pred_values = np.load(os.path.join(loc_path, loc_contents[0]))

In [12]:
pred_values.shape

(1376, 7, 32)

In [13]:
# I think then we have examined the extracted features against the true values - so let's continue this work after food?

In [14]:
# test targets
out_targets_loc = 'data/test_out_targets'
out_contents = os.listdir(out_targets_loc)
out_contents.sort()

In [15]:
out_contents

['test_targets_blk_0103.npy',
 'test_targets_blk_0104.npy',
 'test_targets_blk_0105.npy',
 'test_targets_blk_0106.npy',
 'test_targets_blk_0201.npy',
 'test_targets_blk_0202.npy',
 'test_targets_blk_0205.npy',
 'test_targets_blk_0206.npy',
 'test_targets_blk_0302.npy',
 'test_targets_blk_0303.npy',
 'test_targets_blk_0304.npy',
 'test_targets_blk_0305.npy',
 'test_targets_blk_0306.npy']

In [16]:
# just do this to one npy file
true_targets_blk_0103 = np.load(os.path.join(out_targets_loc, out_contents[0]))

In [17]:
true_targets_blk_0103.shape

(1376, 7, 32)

In [18]:
test_in_0_true = true_targets_blk_0103[0,:,:]

In [19]:
test_in_0_true.shape

(7, 32)

In [20]:
test_in_0_true_df = pd.DataFrame(test_in_0_true)

In [21]:
test_in_0_true_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.000000,0.0,0.000000,0.287550,0.0,0.271238,0.0,0.000000,...,0.000000,0.000000,0.298749,0.0,0.329094,0.00000,0.000000,0.309945,0.287163,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.552956,0.0,0.520200,0.0,0.000000,...,0.000000,0.000000,0.579057,0.0,0.618699,0.00000,0.000000,0.598290,0.543111,0.0
2,0.0,0.0,0.047503,0.0,0.026331,0.031699,0.0,0.027561,0.0,0.025827,...,0.005176,0.054321,0.024045,0.0,0.035481,0.00755,0.034951,0.009745,0.019481,0.0
3,0.0,0.0,0.000000,0.0,0.000000,1.270255,0.0,1.173240,0.0,0.000000,...,0.000000,0.000000,1.313208,0.0,1.364042,0.00000,0.000000,1.367619,1.266346,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.797787,0.0,0.742617,0.0,0.000000,...,0.000000,0.000000,0.829752,0.0,0.874386,0.00000,0.000000,0.877240,0.804976,0.0


In [22]:
# Okay, these values seem much more closer to the values we had when we had the 300,300,3 windows

In [23]:
# display all coumns
with pd.option_context('display.max_columns', None):
    display(test_in_0_true_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.000000,0.0,0.000000,0.287550,0.0,0.271238,0.0,0.000000,0.025975,0.0,0.000000,0.329856,0.000000,0.0,0.0,0.419248,0.0,0.367026,0.331868,0.0,0.000000,0.000000,0.298749,0.0,0.329094,0.00000,0.000000,0.309945,0.287163,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.552956,0.0,0.520200,0.0,0.000000,0.186224,0.0,0.000000,0.636525,0.000000,0.0,0.0,0.761370,0.0,0.685316,0.611933,0.0,0.000000,0.000000,0.579057,0.0,0.618699,0.00000,0.000000,0.598290,0.543111,0.0
2,0.0,0.0,0.047503,0.0,0.026331,0.031699,0.0,0.027561,0.0,0.025827,0.000000,0.0,0.019061,0.025274,0.085747,0.0,0.0,0.044997,0.0,0.037925,0.039310,0.0,0.005176,0.054321,0.024045,0.0,0.035481,0.00755,0.034951,0.009745,0.019481,0.0
3,0.0,0.0,0.000000,0.0,0.000000,1.270255,0.0,1.173240,0.0,0.000000,0.601549,0.0,0.000000,1.498814,0.000000,0.0,0.0,1.600236,0.0,1.534635,1.368480,0.0,0.000000,0.000000,1.313208,0.0,1.364042,0.00000,0.000000,1.367619,1.266346,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.797787,0.0,0.742617,0.0,0.000000,0.344106,0.0,0.000000,0.939440,0.000000,0.0,0.0,1.030106,0.0,0.985694,0.874057,0.0,0.000000,0.000000,0.829752,0.0,0.874386,0.00000,0.000000,0.877240,0.804976,0.0
5,0.0,0.0,0.000000,0.0,0.000000,0.654746,0.0,0.626377,0.0,0.000000,0.244549,0.0,0.000000,0.768023,0.000000,0.0,0.0,0.884145,0.0,0.798062,0.720162,0.0,0.000000,0.000000,0.679187,0.0,0.722182,0.00000,0.000000,0.711515,0.638010,0.0
6,0.0,0.0,0.000000,0.0,0.000000,0.703506,0.0,0.685384,0.0,0.000000,0.317742,0.0,0.000000,0.846666,0.000000,0.0,0.0,0.945741,0.0,0.873481,0.793753,0.0,0.000000,0.000000,0.740951,0.0,0.789030,0.00000,0.000000,0.795199,0.700749,0.0


In [24]:
# Let' not worry too much about the all 0 columns - this is not the case for all dfs, just a few

In [25]:
zero_cols = (test_in_0_true_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 12


In [26]:
# Okay, now what?

In [27]:
# So we do have the extracted features for the test images in each sequence. Now we need the true features for the train images in the sequence for the BLAR model as well. I think that's what we will do next?

In [28]:
# where is this model?
fine_tuned_model = tf.keras.models.load_model("../../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5")

2025-09-10 13:15:47.504964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [29]:
fine_tuned_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [30]:
# Define the feature extractor model

# feature extractor input
feat_ext_input = fine_tuned_model.input

# feature extractor output 
feat_ext_output = fine_tuned_model.layers[-4].output

# define the model
feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [31]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [32]:
# Okay - now what do we need to do?

# I think we do have the inputs stored and arranged in a previous exercise, may be we can use these?

# Where is this location?

sub_windows_of_images_loc = 'data/test_input_sub_images/'

In [33]:
contents_here = os.listdir(sub_windows_of_images_loc)
contents_here.sort()

In [34]:
contents_here

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [35]:
# Let's just try this for a single block, and maybe write a function so that it could be done for the rest?

In [36]:
load_blk_0103_data = np.load(os.path.join(sub_windows_of_images_loc, contents_here[0]))

In [37]:
load_blk_0103_data.shape

(1376, 13, 300, 300, 3)

In [38]:
time_1 = load_blk_0103_data[:,0,:,:,:]

In [39]:
time_1.shape

(1376, 300, 300, 3)

In [40]:
# I think we can get preds for this?
extracted_features_t1 = feature_extractor_model.predict(time_1)

2025-09-10 13:17:01.122847: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


43/43 [==============================] - 6s 17ms/step


In [41]:
extracted_features_t1.shape

(1376, 32)

In [42]:
# just convert this to a df to verify something
extracted_features_t1_df = pd.DataFrame((extracted_features_t1))

In [43]:
extracted_features_t1_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.100598,0.0,0.079017,0.000000,0.0,0.000000,0.031117,0.089639,...,0.099477,0.116435,0.000000,0.032231,0.000000,0.062460,0.078434,0.000000,0.000000,0.0
1,0.0,0.0,0.089276,0.0,0.065604,0.000000,0.0,0.000000,0.016811,0.080069,...,0.085485,0.099877,0.000000,0.020649,0.000000,0.048504,0.067325,0.000000,0.000000,0.0
2,0.0,0.0,0.064533,0.0,0.038549,0.000000,0.0,0.013534,0.000000,0.051904,...,0.059453,0.066543,0.000000,0.000000,0.005930,0.014682,0.042000,0.002515,0.000000,0.0
3,0.0,0.0,0.050633,0.0,0.022977,0.010521,0.0,0.022428,0.000000,0.035801,...,0.039162,0.050234,0.000268,0.000000,0.022586,0.000000,0.027700,0.011048,0.008490,0.0
4,0.0,0.0,0.064971,0.0,0.035631,0.000527,0.0,0.011975,0.000000,0.053071,...,0.054637,0.067611,0.000000,0.000000,0.009256,0.023384,0.042572,0.000000,0.001592,0.0


In [44]:
extracted_features_t1_df.shape

(1376, 32)

In [45]:
# cool, I think now we can go ahead and view all the columns in the dataframe?

In [46]:
# display all coumns
with pd.option_context('display.max_columns', None):
    display(extracted_features_t1_df.head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.100598,0.0,0.079017,0.000000,0.0,0.000000,0.031117,0.089639,0.0,0.0,0.107568,0.000000,0.140781,0.0,0.032884,0.000000,0.0,0.000000,0.000000,0.0,0.099477,0.116435,0.000000,0.032231,0.000000,0.062460,0.078434,0.000000,0.000000,0.0
1,0.0,0.0,0.089276,0.0,0.065604,0.000000,0.0,0.000000,0.016811,0.080069,0.0,0.0,0.093803,0.000000,0.127154,0.0,0.021126,0.000000,0.0,0.000000,0.000000,0.0,0.085485,0.099877,0.000000,0.020649,0.000000,0.048504,0.067325,0.000000,0.000000,0.0
2,0.0,0.0,0.064533,0.0,0.038549,0.000000,0.0,0.013534,0.000000,0.051904,0.0,0.0,0.062787,0.000000,0.097064,0.0,0.000000,0.022186,0.0,0.000000,0.014309,0.0,0.059453,0.066543,0.000000,0.000000,0.005930,0.014682,0.042000,0.002515,0.000000,0.0
3,0.0,0.0,0.050633,0.0,0.022977,0.010521,0.0,0.022428,0.000000,0.035801,0.0,0.0,0.045261,0.014301,0.081678,0.0,0.000000,0.038440,0.0,0.009655,0.027641,0.0,0.039162,0.050234,0.000268,0.000000,0.022586,0.000000,0.027700,0.011048,0.008490,0.0
4,0.0,0.0,0.064971,0.0,0.035631,0.000527,0.0,0.011975,0.000000,0.053071,0.0,0.0,0.059403,0.000570,0.097741,0.0,0.000000,0.022652,0.0,0.000000,0.015493,0.0,0.054637,0.067611,0.000000,0.000000,0.009256,0.023384,0.042572,0.000000,0.001592,0.0


In [47]:
# but notice that over here we do not have all zero columns - make sense we have not concatenated data in a time direction yet
zero_cols = (extracted_features_t1_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 0


In [48]:
# see, there are any columns will all zeros, so we should be all okay for the BLAR model

In [49]:
# load_blk_0103_data.shape[1]

In [50]:
# Let's get the predictions across all time points in a for loop?
catch_all_preds_block_0103 = []
for i in range(load_blk_0103_data.shape[1]):
    time_wise_data = load_blk_0103_data[:,i,:,:,:]
    extracted_features = feature_extractor_model.predict(time_wise_data)
    catch_all_preds_block_0103.append(extracted_features)

43/43 [==============================] - 1s 16ms/step


In [51]:
# check the shape of the prediction arrays?

In [52]:
for pred_array in catch_all_preds_block_0103:
    print(pred_array.shape)

(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)
(1376, 32)


In [53]:
# stack all these together? - maybe to be of shape 910, 13, 32
stacked_features_0103 = np.stack(catch_all_preds_block_0103, axis = 1)

In [54]:
stacked_features_0103.shape

(1376, 13, 32)

In [55]:
sub_0 = stacked_features_0103[0,:,:]

In [56]:
sub_0_df = pd.DataFrame(sub_0)

In [57]:
sub_0_df.shape

(13, 32)

In [58]:
with pd.option_context('display.max_columns', None):
    display(sub_0_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.000000,0.000000,0.100598,0.000000,0.079017,0.000000,0.0,0.000000,0.031117,0.089639,0.000000,0.000000,0.107568,0.000000,0.140781,0.000000,0.032884,0.000000,0.000000,0.000000,0.000000,0.000000,0.099477,0.116435,0.000000,0.032231,0.000000,0.062460,0.078434,0.000000,0.000000,0.000000
1,0.000000,0.013985,0.101657,0.000000,0.077531,0.000000,0.0,0.000000,0.044242,0.099687,0.000000,0.000000,0.110163,0.000000,0.147671,0.030575,0.057605,0.000000,0.000000,0.000000,0.000000,0.000000,0.109002,0.118194,0.000000,0.044611,0.000000,0.076928,0.087441,0.000000,0.000000,0.000000
2,0.000000,0.025038,0.087745,0.000000,0.079452,0.000000,0.0,0.000000,0.042732,0.099607,0.000000,0.000000,0.110106,0.000000,0.134006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.114905,0.106012,0.000000,0.037057,0.000000,0.021749,0.075469,0.000000,0.000000,0.000000
3,0.103946,0.147407,0.204129,0.098371,0.188013,0.000000,0.0,0.000000,0.160088,0.198620,0.000000,0.098426,0.234429,0.000000,0.265795,0.156740,0.156941,0.000000,0.079863,0.000000,0.000000,0.145643,0.227564,0.243455,0.000000,0.163562,0.000000,0.190030,0.185727,0.000000,0.000000,0.059823
4,0.017576,0.085960,0.171267,0.033650,0.147210,0.000000,0.0,0.000000,0.112189,0.165176,0.000000,0.081027,0.185482,0.000000,0.224070,0.122870,0.113545,0.000000,0.013965,0.000000,0.000000,0.100588,0.179570,0.208353,0.000000,0.117695,0.000000,0.171195,0.155618,0.000000,0.000000,0.000000
5,0.000000,0.017385,0.151121,0.000000,0.127584,0.000000,0.0,0.000000,0.077782,0.127061,0.000000,0.012600,0.150414,0.000000,0.201345,0.078723,0.075640,0.000000,0.000000,0.000000,0.000000,0.014458,0.125769,0.184329,0.000000,0.060056,0.000000,0.139587,0.133328,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.179295,0.0,0.173787,0.000000,0.000000,0.000000,0.000000,0.000000,0.197154,0.000000,0.000000,0.000000,0.277654,0.000000,0.226825,0.214288,0.000000,0.000000,0.000000,0.176298,0.000000,0.197388,0.000000,0.000000,0.188467,0.168912,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.146837,0.0,0.142099,0.000000,0.000000,0.000000,0.000000,0.000000,0.171031,0.000000,0.000000,0.000000,0.216890,0.000000,0.193989,0.179075,0.000000,0.000000,0.000000,0.145257,0.000000,0.171121,0.000000,0.000000,0.149026,0.139806,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.604147,0.0,0.571729,0.000000,0.000000,0.217120,0.000000,0.000000,0.719837,0.000000,0.000000,0.000000,0.824087,0.000000,0.740439,0.666596,0.000000,0.000000,0.000000,0.628526,0.000000,0.671062,0.000000,0.000000,0.649773,0.606993,0.000000
9,0.000000,0.000000,0.015217,0.000000,0.000000,0.039908,0.0,0.045248,0.000000,0.000000,0.000000,0.000000,0.000000,0.037591,0.055842,0.000000,0.000000,0.075882,0.000000,0.045005,0.051835,0.000000,0.000000,0.039709,0.027653,0.000000,0.042844,0.000000,0.006355,0.028997,0.030003,0.000000


In [59]:
zero_cols = (sub_0_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 1


In [60]:
# There are all 0 columns - but is this true for all sub windows?

In [61]:
sub_98 = stacked_features_0103[98,:,:]

In [62]:
sub_98_df = pd.DataFrame(sub_98)

In [63]:
with pd.option_context('display.max_columns', None):
    display(sub_98_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.000000,0.018986,0.120683,0.000000,0.087005,0.000000,0.0,0.000000,0.053708,0.100685,0.0,0.000000,0.115726,0.000000,0.149990,0.032734,0.066198,0.000000,0.000000,0.000000,0.000000,0.000000,0.114132,0.134362,0.000000,0.048163,0.000000,0.085725,0.103510,0.000000,0.000000,0.000000
1,0.000000,0.001913,0.121510,0.000000,0.065510,0.000000,0.0,0.000000,0.021753,0.100557,0.0,0.001312,0.109880,0.000000,0.153289,0.074384,0.072128,0.000000,0.000000,0.000000,0.000000,0.027486,0.093175,0.135177,0.000000,0.035880,0.000000,0.106131,0.101025,0.000000,0.000000,0.000000
2,0.098723,0.145770,0.197803,0.070187,0.174016,0.000000,0.0,0.000000,0.153617,0.188534,0.0,0.068218,0.225948,0.000000,0.251929,0.145057,0.153648,0.000000,0.059253,0.000000,0.000000,0.110964,0.209963,0.228337,0.000000,0.145054,0.000000,0.174431,0.176822,0.000000,0.000000,0.033098
3,0.114050,0.152359,0.219730,0.105945,0.200510,0.000000,0.0,0.000000,0.170759,0.216252,0.0,0.108196,0.248602,0.000000,0.284132,0.177316,0.164981,0.000000,0.100572,0.000000,0.000000,0.171485,0.236747,0.253656,0.000000,0.168916,0.000000,0.208342,0.197784,0.000000,0.000000,0.077704
4,0.000000,0.012629,0.121849,0.000000,0.081928,0.000000,0.0,0.000000,0.052952,0.102644,0.0,0.029580,0.118606,0.000000,0.163051,0.071448,0.060251,0.000000,0.000000,0.000000,0.000000,0.017797,0.102652,0.151757,0.000000,0.036157,0.000000,0.118249,0.108486,0.000000,0.000000,0.000000
5,0.115746,0.155302,0.221302,0.109961,0.205728,0.000000,0.0,0.000000,0.172300,0.218461,0.0,0.107786,0.257177,0.000000,0.294310,0.177017,0.163987,0.000000,0.099631,0.000000,0.000000,0.167927,0.243392,0.264786,0.000000,0.176367,0.000000,0.207562,0.199051,0.000000,0.000000,0.082874
6,0.000000,0.044577,0.153382,0.000000,0.111401,0.000000,0.0,0.000000,0.075789,0.129044,0.0,0.044043,0.152868,0.000000,0.188448,0.093991,0.091421,0.000000,0.000000,0.000000,0.000000,0.041911,0.132264,0.183209,0.000000,0.069555,0.000000,0.144867,0.132894,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.068123,0.0,0.076127,0.000000,0.000000,0.0,0.000000,0.000000,0.076743,0.008200,0.000000,0.000000,0.099773,0.000000,0.086133,0.091701,0.000000,0.000000,0.000000,0.054452,0.000000,0.080540,0.000000,0.000000,0.067207,0.061216,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.090766,0.0,0.098964,0.000000,0.000000,0.0,0.000000,0.000000,0.115213,0.000000,0.000000,0.000000,0.142188,0.000000,0.115264,0.117436,0.000000,0.000000,0.000000,0.083969,0.000000,0.112375,0.000000,0.000000,0.097506,0.082405,0.000000
9,0.000000,0.018799,0.134244,0.000000,0.103982,0.000000,0.0,0.000000,0.064693,0.118241,0.0,0.000000,0.127951,0.000000,0.169761,0.033114,0.061970,0.000000,0.000000,0.000000,0.000000,0.000000,0.122798,0.150459,0.000000,0.059058,0.000000,0.109980,0.114798,0.000000,0.000000,0.000000


In [64]:
# Are there any 0 all columns?
zero_cols = (sub_98_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 2


In [65]:
stacked_features_0103.shape

(1376, 13, 32)

In [66]:
train_save_path = 'data/train_features_non_overlapping/'

In [67]:
np.save(os.path.join(train_save_path, 'train_features_block_0103.npy'), stacked_features_0103)

In [68]:
san_check_blk_0103 = np.load('data/train_features_non_overlapping/train_features_block_0103.npy')

In [69]:
np.mean(stacked_features_0103 == san_check_blk_0103)

1.0

In [70]:
# Okay, so let's define a function for this
sub_windows_of_images_loc

'data/test_input_sub_images/'

In [71]:
train_save_path

'data/train_features_non_overlapping/'

In [72]:
def store_train_extracted_features(path_to_inputs, input_feature_file, save_path):
    # load the file
    loaded_input_file = np.load(os.path.join(path_to_inputs, input_feature_file))
    # Let's get the predictions across all time points in a for loop?
    catch_all_preds = []
    for i in range(loaded_input_file.shape[1]):
        time_wise_data = loaded_input_file[:,i,:,:,:]
        extracted_features = feature_extractor_model.predict(time_wise_data)
        catch_all_preds.append(extracted_features)

    # stack these predictions?
    stacked_features = np.stack(catch_all_preds, axis = 1)
    # save the stack of extracted features?
    save_name = 'train_features_block_' + input_feature_file.split('.')[0][-4:] + '.npy'
    np.save(os.path.join(save_path, save_name), stacked_features)
    # also do the sanity check?
    print(np.mean(np.load(os.path.join(save_path, save_name)) == stacked_features))
    return stacked_features
    

In [ ]:
# see if this works for block 0103?
stack_0103 = store_train_extracted_features(sub_windows_of_images_loc, contents_here[0], train_save_path)

In [ ]:
# Seems to be working, do this to the rest of the blocks?

In [ ]:
%%time
# Easier to do it in a for loop - but verify this tomorrow
all_stacks = []
for i in range(len(contents_here)):
    stack = store_train_extracted_features(sub_windows_of_images_loc, contents_here[i], train_save_path)
    all_stacks.append(stack)
    

In [1]:
# Cannot get this small lag at the end to work - may have to submit this as a job too?